# Samples cleaning

This task will fetch the raw data from the remote database given a SQL query file.

## Import libraries

In [1]:
import sys

sys.path.append('..')

In [2]:
import os
import gc
import yaml
import json
import requests
import numpy as np
import pandas as pd

from pymongo import MongoClient

from utils import get_files, memory_usage, \
correct_encoding, update_page_metadata

## Load the data

In [3]:
BASE_PATH = '../../'

CONFIG_DIR = os.path.join(BASE_PATH, 'config')
STORAGE_DIR = os.path.join(BASE_PATH, 'storage')
DATA_DIR = os.path.join(BASE_PATH, 'data')

config = yaml.load(open(os.path.join(CONFIG_DIR, 'env.yml')),
                   Loader=yaml.FullLoader)

In [4]:
model = 'samples'
model_storage = os.path.join(STORAGE_DIR, 'samples-big')

if not os.path.exists(model_storage):
    os.makedirs(model_storage)
    
client = MongoClient('mongodb://{}:{}@{}:{}'.format(config['MONGO_USERNAME'], config['MONGO_PASSWORD'],
                                                    config['MONGO_HOST'], config['MONGO_PORT']))
metadata_db = client[config['MONGO_DATABASE']]

model_metadata = metadata_db[model]

model_metadata.delete_many({})
model_metadata.insert_one({'pages': []})

In [5]:
csv_files = get_files(os.path.join(DATA_DIR, 'raw', model, '*.csv'))

In [6]:
if not csv_files:
    raise FileNotFoundError('Couldn\'t find any csv files! Please make sure the filepath exists')

In [7]:
df = pd.read_csv(csv_files[0], sep=';', parse_dates=['timestamp'], nrows=100)

# sample rows to list columns and dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    100 non-null    int64         
 1   device_id             100 non-null    int64         
 2   timestamp             100 non-null    datetime64[ns]
 3   battery_state         100 non-null    object        
 4   battery_level         100 non-null    float64       
 5   timezone              100 non-null    object        
 6   country_code          100 non-null    object        
 7   charger               100 non-null    object        
 8   health                100 non-null    object        
 9   voltage               100 non-null    float64       
 10  temperature           100 non-null    float64       
 11  usage                 100 non-null    float64       
 12  up_time               100 non-null    int64         
 13  sleep_time           

In [8]:
df.describe(include=[np.number])

,id,device_id,battery_level,voltage,temperature,usage,up_time,sleep_time,wifi_signal_strength,wifi_link_speed,...,roaming_enabled,bluetooth_enabled,location_enabled,power_saver_enabled,nfc_enabled,developer_mode,free,total,free_system,total_system
count,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,1.000000e+02,1.000000e+02,100.000000,100.000000,...,100.0,100.0,100.000000,100.0,100.000000,100.0,100.000000,100.000000,100.000000,100.000000
mean,50.500000,1.870000,71.380000,4.119500,28.77100,0.320900,6.850256e+05,4.672100e+05,-107.430000,14.610000,...,0.0,0.0,0.710000,0.0,0.290000,0.0,4098.300000,10567.510000,460.100000,2699.510000
std,29.011492,1.368144,31.909208,0.181578,4.09235,2.246707,4.056857e+05,5.102206e+05,27.901434,28.977837,...,0.0,0.0,0.456048,0.0,0.456048,0.0,332.706001,1468.018581,278.189293,1085.850339
min,1.000000,1.000000,5.000000,3.790000,21.40000,-21.430000,4.086870e+05,1.413690e+05,-127.000000,-1.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,3789.000000,9634.000000,27.000000,1009.000000
25%,25.750000,1.000000,40.750000,3.930000,27.30000,0.030000,4.130460e+05,1.415370e+05,-127.000000,-1.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,3919.000000,9634.000000,27.000000,1009.000000
50%,50.500000,1.000000,88.000000,4.175000,27.70000,0.695000,4.254800e+05,1.436480e+05,-127.000000,-1.000000,...,0.0,0.0,1.000000,0.0,0.000000,0.0,3921.000000,9634.000000,637.000000,3390.000000
75%,75.250000,4.000000,94.000000,4.290000,32.02500,0.875000,1.314802e+06,1.260494e+06,-86.750000,1.250000,...,0.0,0.0,1.000000,0.0,1.000000,0.0,4611.000000,12853.000000,637.000000,3390.000000
max,100.000000,4.000000,100.000000,4.320000,37.20000,2.210000,1.317836e+06,1.263017e+06,-54.000000,72.000000,...,0.0,0.0,1.000000,0.0,1.000000,0.0,4611.000000,12853.000000,637.000000,3390.000000


In [9]:
df.describe(exclude=[np.number])

,timestamp,battery_state,timezone,country_code,charger,health,network_status,mobile_data_status,mobile_data_activity,wifi_status
count,100,100,100,100,100,100,100,100,100,100
unique,98,3,2,2,2,1,3,2,3,1
top,2017-10-08 12:50:04,Charging,America/Chicago,us,ac,Good,lte,disconnected,none,enabled
freq,2,74,71,71,76,100,36,64,57,100
first,2017-10-08 12:50:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2017-10-09 03:51:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Basic data pre-processing

In [10]:
def save_df_page(page, chunks, collection, exclude=['id']):
    # concatenate data chunks -> careful benchmark this
    df = pd.concat(chunks, axis=0, ignore_index=True)
    
    collection.update_one({}, {'$inc': {'nrows': df.shape[0]}})

    # store page metadata in mongo
    update_page_metadata(collection, df)

    # save output to a parquet file with brotli compression
    df.to_parquet(os.path.join(model_storage, '{}.{}.parquet'.format(
        model, page)), compression='brotli', index=False)
    
    del df

    return page + 1

In [11]:
# projected memory usage for all samples loaded
round(memory_usage(df) * 5000000 / df.shape[0], 2)

3500.0

In [12]:
mappings = {'id': 'uint32', 'device_id': 'uint32', 'battery_level': 'uint8', 'usage': 'uint8',
            'screen_on': 'bool', 'wifi_signal_strength': 'int16', 'wifi_link_speed': 'int16',
            'screen_brightness': 'int16', 'roaming_enabled': 'bool', 'bluetooth_enabled': 'bool',
            'location_enabled': 'bool', 'power_saver_enabled': 'bool', 'nfc_enabled': 'bool',
            'developer_mode': 'bool', 'free': 'uint32', 'total': 'uint32', 'free_system': 'uint32',
            'total_system': 'uint32'}

model_metadata.update_one({}, {'$set': {'mappings': mappings}})

In [13]:
total_rows = 0
total_memory_usage = 0.0
memory_usage_split = 0.0
last_id = 0

page = 0
chunks = []

features = df.columns.to_list()
features = list(filter(lambda x: x not in ['country_code'], features))

In [14]:
print('Total csv files: {}'.format(len(csv_files)))

Total csv files: 165


In [ ]:
for filepath in csv_files:
    df = pd.read_csv(filepath, sep=';', usecols=features,
                     parse_dates=['timestamp'])

    # drop missing values
    df = df.dropna()

    # process features which can drop values first
    df = df[df['timestamp'].between(pd.Timestamp(
        2017, 9, 1), pd.Timestamp(2020, 8, 1))]

    # battery level
    df = df[df['battery_level'] <= 100]

    # temperature
    lower, upper = df['temperature'].quantile([0.01, 0.99])
    df = df[df['temperature'].between(lower, upper)]

    # voltage
    df.loc[df['voltage'] > 1000, 'voltage'] = df['voltage'] / 1000

    # cpu usage
    df.loc[:, 'usage'] = abs(df['usage']) * 100
    df.loc[df['usage'] > 100, 'usage'] = df['usage'] / 100
    df = df.drop(df[df['usage'] > 100].index, axis=0)

    string_columns = df.select_dtypes(include='object').columns.to_list()

    # basic string pre-processing: convert to lowercase and strip blank chars
    for column in string_columns:
        df.loc[:, column] = [x.lower().strip() for x in df[column].tolist()]

    df[string_columns] = df[string_columns].astype('category')
    df = df.astype(mappings)

    total_rows += df.shape[0]
    page_memory_usage = memory_usage(df)
    total_memory_usage += page_memory_usage
    memory_usage_split += page_memory_usage

    last_id = max(last_id, df['id'].max())
    
    chunks.append(df)

    if memory_usage_split >= config['MEMORY_USAGE_SPLIT']:
        print('Page {} created!'.format(page))
        page = save_df_page(page, chunks, model_metadata)

        del chunks[:]
        del chunks
        gc.collect()
        
        chunks = []
        memory_usage_split = 0.0

    print('{} ✔'.format(filepath))

../../data/raw/samples/samples.query.1.csv ✔
../../data/raw/samples/samples.query.2.csv ✔
../../data/raw/samples/samples.query.3.csv ✔
../../data/raw/samples/samples.query.4.csv ✔
../../data/raw/samples/samples.query.5.csv ✔
../../data/raw/samples/samples.query.6.csv ✔
../../data/raw/samples/samples.query.7.csv ✔
../../data/raw/samples/samples.query.8.csv ✔
../../data/raw/samples/samples.query.9.csv ✔
../../data/raw/samples/samples.query.10.csv ✔
../../data/raw/samples/samples.query.11.csv ✔
../../data/raw/samples/samples.query.12.csv ✔
../../data/raw/samples/samples.query.13.csv ✔
../../data/raw/samples/samples.query.14.csv ✔
../../data/raw/samples/samples.query.15.csv ✔
../../data/raw/samples/samples.query.16.csv ✔
../../data/raw/samples/samples.query.17.csv ✔


In [ ]:
if chunks:
    save_df_page(page, chunks, model_metadata)
    del chunks[:]
    del chunks
    gc.collect()

In [ ]:
model_metadata.update_one({}, {'$set': {'last_id': int(last_id)}})
model_metadata.update_one({}, {'$set': {'total': total_rows}})

In [ ]:
total_rows, total_memory_usage, last_id

In [ ]:
content = '**{}** | task: data pre-processing, rows: {}, memory usage: {} MB' \
.format(model, total_rows, total_memory_usage)

requests.post(config['discord_webhook_url'], data={'content': content})